In [2]:
import pandas as pd

In [3]:
data=pd.read_csv("spam.csv",encoding="latin-1")
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [4]:
train_data=data[['v1','v2']]
train_data

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [5]:
train_data.loc[:, 'v1'] = train_data['v1'].map({'ham': 0, 'spam': 1})

In [6]:
train_data

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [7]:
import os
import re
import math
from collections import defaultdict, Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

class NaiveBayesSpamClassifier:
    def __init__(self):
        self.spam_word_probs = defaultdict(float)
        self.ham_word_probs = defaultdict(float)
        self.spam_prior = 0.5 

    def extract_features(self, email_content):
        words = re.findall(r'\b\w+\b', email_content.lower())
        word_counts = Counter(words)
        num_links = len(re.findall(r'http[s]?://', email_content))
        num_special_chars = len(re.findall(r'[!$%&]', email_content))
        features = {
            'num_words': len(words),
            'num_links': num_links,
            'num_special_chars': num_special_chars,
            **word_counts
        }
        return features

    def train(self, emails, labels):
        spam_counts = defaultdict(int)
        ham_counts = defaultdict(int)
        total_spam_words = total_ham_words = 0

        for email, label in zip(emails, labels):
            word_counts = self.extract_features(email)
            if label == 'spam':
                for word, count in word_counts.items():
                    spam_counts[word] += count
                    total_spam_words += count
            else:
                for word, count in word_counts.items():
                    ham_counts[word] += count
                    total_ham_words += count

        # Laplace smoothing
        self.spam_word_probs = {
            word: (count + 1) / (total_spam_words + len(spam_counts))
            for word, count in spam_counts.items()
        }
        self.ham_word_probs = {
            word: (count + 1) / (total_ham_words + len(ham_counts))
            for word, count in ham_counts.items()
        }

    def predict(self, email):
        word_counts = self.extract_features(email)
        spam_score = math.log(self.spam_prior)
        ham_score = math.log(1 - self.spam_prior)

        for word, count in word_counts.items():
            spam_score += count * math.log(self.spam_word_probs.get(word, 1e-6))
            ham_score += count * math.log(self.ham_word_probs.get(word, 1e-6))

        return 'spam' if spam_score > ham_score else 'ham'

def classify_emails_from_folder(folder_path, classifier):
    results = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r') as f:
                email_content = f.read()
                prediction = classifier.predict(email_content)
                results.append((filename, prediction))
                print(f"{filename}: {prediction.capitalize()}")
    return results


emails = train_data['v2'].tolist()
labels = train_data['v1'].tolist()
X_train, X_test, y_train, y_test = train_test_split(emails, labels, test_size=0.2, random_state=42)

classifier = NaiveBayesSpamClassifier()
classifier.train(X_train, y_train)

y_pred = [classifier.predict(email) for email in X_test]

print(classification_report(y_test, y_pred, target_names=["ham", "spam"]))
print("Accuracy:" ,accuracy_score(y_test,y_pred))
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred))

print("\nClassifying emails from 'test' folder:")

test_results = classify_emails_from_folder('test', classifier)


ValueError: Mix of label input types (string and number)

In [55]:
import os
import re
import math
import pandas as pd
from collections import defaultdict, Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

class NaiveBayesSpamClassifier:
    def __init__(self):
        self.spam_word_probs = defaultdict(float)
        self.ham_word_probs = defaultdict(float)
        self.spam_prior = 0.5 

    def extract_features(self, email_content):
        words = re.findall(r'\b\w+\b', email_content.lower())
        word_counts = Counter(words)
        num_links = len(re.findall(r'http[s]?://', email_content))
        num_special_chars = len(re.findall(r'[!$%&]', email_content))
        features = {
            'num_words': len(words),
            'num_links': num_links,
            'num_special_chars': num_special_chars,
            **word_counts
        }
        return features

    def train(self, emails, labels):
        spam_counts = defaultdict(int)
        ham_counts = defaultdict(int)
        total_spam_words = total_ham_words = 0

        for email, label in zip(emails, labels):
            word_counts = self.extract_features(email)
            if label == 1:  # Spam
                for word, count in word_counts.items():
                    spam_counts[word] += count
                    total_spam_words += count
            else:  # Ham
                for word, count in word_counts.items():
                    ham_counts[word] += count
                    total_ham_words += count

        # Laplace smoothing
        self.spam_word_probs = {
            word: (count + 1) / (total_spam_words + len(spam_counts))
            for word, count in spam_counts.items()
        }
        self.ham_word_probs = {
            word: (count + 1) / (total_ham_words + len(ham_counts))
            for word, count in ham_counts.items()
        }

    def predict(self, email):
        word_counts = self.extract_features(email)
        spam_score = math.log(self.spam_prior)
        ham_score = math.log(1 - self.spam_prior)

        for word, count in word_counts.items():
            spam_score += count * math.log(self.spam_word_probs.get(word, 1e-6))
            ham_score += count * math.log(self.ham_word_probs.get(word, 1e-6))

        return +1 if spam_score > ham_score else 0

def classify_emails_from_folder(folder_path, classifier):
    results = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as f:
                email_content = f.read()
                prediction = classifier.predict(email_content)
                results.append((filename, prediction))
                print(f"{filename}: {'Spam' if prediction == 1 else 'Non-Spam'} ({prediction})")
    return results


train_data.loc[:, 'v1'] = train_data['v1'].map({'ham': 0, 'spam': 1})


emails = train_data['v2'].tolist()
labels = train_data['v1'].tolist()

X_train, X_test, y_train, y_test = train_test_split(emails, labels, test_size=0.2, random_state=42)

classifier = NaiveBayesSpamClassifier()
classifier.train(X_train, y_train)
y_pred = [classifier.predict(email) for email in X_test]

print("Classification Report:\n", classification_report(y_test, y_pred, target_names=["Non-Spam", "Spam"]))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

print("\nClassifying emails from 'test' folder:")
test_results = classify_emails_from_folder("./test", classifier)


C:\Users\Rajnish\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\_array_api.py:390: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


ValueError: Input y_true contains NaN.

In [77]:
import os
import re
import math
import pandas as pd
from collections import defaultdict, Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

class NaiveBayesSpamClassifier:
    def __init__(self):
        self.spam_word_probs = defaultdict(float)
        self.ham_word_probs = defaultdict(float)
        self.spam_prior = 0.5 

    def extract_features(self, email_content):
        words = re.findall(r'\b\w+\b', email_content.lower())
        word_counts = Counter(words)
        num_links = len(re.findall(r'http[s]?://', email_content))
        num_special_chars = len(re.findall(r'[!$%&]', email_content))
        features = {
            'num_words': len(words),
            'num_links': num_links,
            'num_special_chars': num_special_chars,
            **word_counts
        }
        return features

    def train(self, emails, labels):
        spam_counts = defaultdict(int)
        ham_counts = defaultdict(int)
        total_spam_words = total_ham_words = 0

        for email, label in zip(emails, labels):
            word_counts = self.extract_features(email)
            if label == 1:  # Spam
                for word, count in word_counts.items():
                    spam_counts[word] += count
                    total_spam_words += count
            else:  # Ham
                for word, count in word_counts.items():
                    ham_counts[word] += count
                    total_ham_words += count

        # Laplace smoothing
        self.spam_word_probs = {
            word: (count + 1) / (total_spam_words + len(spam_counts))
            for word, count in spam_counts.items()
        }
        self.ham_word_probs = {
            word: (count + 1) / (total_ham_words + len(ham_counts))
            for word, count in ham_counts.items()
        }

    def predict(self, email):
        word_counts = self.extract_features(email)
        spam_score = math.log(self.spam_prior)
        ham_score = math.log(1 - self.spam_prior)

        for word, count in word_counts.items():
            spam_score += count * math.log(self.spam_word_probs.get(word, 1e-6))
            ham_score += count * math.log(self.ham_word_probs.get(word, 1e-6))

        return +1 if spam_score > ham_score else 0

def classify_emails_from_folder(folder_path, classifier):
    results = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as f:
                email_content = f.read()
                prediction = classifier.predict(email_content)
                results.append((filename, prediction))
                print(f"{filename}: {'Spam' if prediction == 1 else 'Non-Spam'} ({prediction})")
    return results



emails = train_data['v2'].tolist()
labels = train_data['v1'].tolist()

X_train, X_test, y_train, y_test = train_test_split(emails, labels, test_size=0.2, random_state=42)

classifier = NaiveBayesSpamClassifier()
classifier.train(X_train, y_train)

y_pred = [classifier.predict(email) for email in X_test]

print("Classification Report:\n", classification_report(y_test, y_pred, target_names=["Non-Spam", "Spam"]))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

test_folder_path = "M.Tech DSAI 2024-2026/Semester 1/FML(FOUNDATION OF MACHINE LEARNING)/Assignments/Assignment2/test"

print("\nClassifying emails from 'test' folder:")
test_results = classify_emails_from_folder(test_folder_path, classifier)


Classification Report:
               precision    recall  f1-score   support

    Non-Spam       0.99      1.00      0.99       965
        Spam       0.97      0.91      0.94       150

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.97      1115
weighted avg       0.98      0.98      0.98      1115

Accuracy: 0.9847533632286996
Confusion Matrix:
 [[961   4]
 [ 13 137]]

Classifying emails from 'test' folder:


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'M.Tech DSAI 2024-2026/Semester 1/FML(FOUNDATION OF MACHINE LEARNING)/Assignments/Assignment2/test'

In [9]:
import os
import re
import math
import pandas as pd
from collections import defaultdict, Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

class NaiveBayesSpamClassifier:
    def __init__(self):
        self.spam_word_probs = defaultdict(float)
        self.ham_word_probs = defaultdict(float)
        self.spam_prior = 0.5

    def extract_features(self, email_content):
        words = re.findall(r'\b\w+\b', email_content.lower())
        word_counts = Counter(words)
        num_links = len(re.findall(r'http[s]?://', email_content))
        num_special_chars = len(re.findall(r'[!$%&]', email_content))
        features = {
            'num_words': len(words),
            'num_links': num_links,
            'num_special_chars': num_special_chars,
            **word_counts
        }
        return features

    def train(self, emails, labels):
        spam_counts = defaultdict(int)
        ham_counts = defaultdict(int)
        total_spam_words = total_ham_words = 0

        for email, label in zip(emails, labels):
            word_counts = self.extract_features(email)
            if label == 1:  # Spam
                for word, count in word_counts.items():
                    spam_counts[word] += count
                    total_spam_words += count
            else:  # Ham
                for word, count in word_counts.items():
                    ham_counts[word] += count
                    total_ham_words += count

        # Laplace smoothing
        self.spam_word_probs = {
            word: (count + 1) / (total_spam_words + len(spam_counts))
            for word, count in spam_counts.items()
        }
        self.ham_word_probs = {
            word: (count + 1) / (total_ham_words + len(ham_counts))
            for word, count in ham_counts.items()
        }

    def predict(self, email):
        word_counts = self.extract_features(email)
        spam_score = math.log(self.spam_prior)
        ham_score = math.log(1 - self.spam_prior)

        for word, count in word_counts.items():
            spam_score += count * math.log(self.spam_word_probs.get(word, 1e-6))
            ham_score += count * math.log(self.ham_word_probs.get(word, 1e-6))

        return +1 if spam_score > ham_score else 0

def classify_emails_from_folder(folder_path, classifier):
    results = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as f:
                email_content = f.read()
                prediction = classifier.predict(email_content)
                results.append((filename, prediction))
                print(f"{filename}: {'Spam' if prediction == 1 else 'Non spam'} ({prediction})")
    return results

emails = train_data['v2'].tolist()
labels = train_data['v1'].tolist()

X_train, X_test, y_train, y_test = train_test_split(emails, labels, test_size=0.2, random_state=42)

classifier = NaiveBayesSpamClassifier()
classifier.train(X_train, y_train)

y_pred = [classifier.predict(email) for email in X_test]
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=["Non-Spam", "Spam"]))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

test_folder_path = "test"

if os.path.exists(test_folder_path):
    print("\nClassifying emails from 'test' folder:")
    test_results = classify_emails_from_folder(test_folder_path, classifier)
else:
    print("Error: The specified folder path does not exist. Please check the path.")


Classification Report:
               precision    recall  f1-score   support

    Non-Spam       0.99      1.00      0.99       965
        Spam       0.97      0.91      0.94       150

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.97      1115
weighted avg       0.98      0.98      0.98      1115

Accuracy: 0.9847533632286996
Confusion Matrix:
 [[961   4]
 [ 13 137]]

Classifying emails from 'test' folder:
email1.txt: Spam (1)
email2.txt: Non spam (0)
email3.txt: Non spam (0)
